In [ ]:
import dask
import datetime
import matplotlib.pyplot as plt
import xarray as xr

from pathlib import PurePath, Path
from dask_cluster import CHPC, start_cluster

In [ ]:
home = PurePath(f'{Path.home()}/scratch/iSnobal/runs/run201[7,8]*')
snow_nc = 'snow.nc'

In [ ]:
start_cluster(cores=8, memory=12)

In [ ]:
snow = xr.open_mfdataset(
    home.joinpath(snow_nc).as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=datetime.time(23))

In [ ]:
def plot_day(snow_day):
    plt.figure(figsize=(10, 10), dpi=300, tight_layout=True)
    snow_day.plot(
        cmap=plt.cm.Blues, vmax=3, 
        cbar_kwargs={'label': 'Snow Height'}
    )
    plt.xlabel('')
    plt.ylabel('')
    plt.gca().ticklabel_format(style='plain')
    plt.title(snow_day.time.dt.strftime('%Y-%m-%d').values)
    plt.savefig(f'{Path.home()}/scratch/iSnobal/plots/{str(snow_day.time.dt.strftime("%y%m%d").values)}.jpg')
    plt.close()

In [ ]:
dask_queue = []

for snow_day in snow.thickness:
    dask_queue.append(
        dask.delayed(plot_day)(snow_day)
    )

In [ ]:
dask.compute(dask_queue);

### Convert to video using ffmpeg

```shell
ffmpeg -pattern_type glob -i '*.jpg' -crf 18 -filter:v "setpts=2.5*PTS" snow_depth.mp4
```